In [1]:
# 和 ngrok 帳戶進行綁定
!ngrok authtoken 2igYHzOXqOKqwMdqwf6EajHCZgV_5gw6SAov6sUo7ewGD3Dzb

Authtoken saved to configuration file: C:\Users\lenovo\AppData\Local/ngrok/ngrok.yml


In [2]:
import getpass
from pyngrok import ngrok, conf


#輸入 ngrok token 並與 ngrok 進行認證。
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect("22", "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

#2icxQFAqPrkPqGoa8jo4yED1DEW_4u4CbtfkK3Y9wZn3B67Kp

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
 * ngrok tunnel available, access with `ssh root@6.tcp.ngrok.io -p15722`


In [3]:
from flask import Flask, request, abort
from pyngrok import ngrok
import joblib
import pandas as pd
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ButtonsTemplate, PostbackAction, PostbackEvent

app = Flask(__name__)
port = 5000

# Set up ngrok tunnel to expose local server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ") # 印出 tunnel

# LINE Messaging API credentials
# access_token = 'hr57G9x2N1RzDTLfuTmZcmEGbp4NOE+RhKy9bpYVAmJh8kkdmOxjjI2XyyJ1pL2G7fuT0dFbzu7tN/9CXcZtJM6r9n7XLE5ftm04RyX1mNoQ36SLxG+aZo6cGOQe0RUNcsPDNT2Jr+/0THQ4SboBuwdB04t89/1O/w1cDnyilFU='
# secret = '55bd2bf40d93d598909c39d480562ddd'
access_token = '5wVS/iUy2WzX8Xgbb8UMR1SRjRiFUTZ/hzxY7VyVOhlqL67rJEBli2hjFn0cnDuOu/cy/TjTcF7a7uVEoIe5qzVK1UDTu0NJtcWjO0Y+AYxTYrrDNZwnZZhQC9FQzPosJO6pHH5RrMTEA3paoVoC4AdB04t89/1O/w1cDnyilFU='
secret = 'c4a775434285e5044364bfc085a0af68'
line_bot_api = LineBotApi(access_token)  # token 確認
handler = WebhookHandler(secret)      # secret 確認

model = joblib.load('./diabete_prediction_model.pkl')

# User state management
user_state = {}

# Text questions (4-7)
questions = [
    "請輸入年齡: ",
    "請輸入BMI: ",
    "請輸入HbA1c水平: ",
    "請輸入血糖水平: "
]

# Button questions (0-3)
button_questions = {
    0: ('性別', [
        ('男', '0'),
        ('女', '1'),
        ('其他', '2')
    ]),
    1: ('高血壓狀況', [
        ('有', '1'),
        ('無', '0')
    ]),
    2: ('心臟病狀況', [
        ('有', '1'),
        ('無', '0')
    ]),
    3: ('吸煙習慣', [
        ('ever', '4'),
        ('current', '3'),
        ('former', '1'),
        ('never', '0')
    ])
}

@app.route("/", methods=['POST'])
def webhook():
    body = request.get_data(as_text=True)
    signature = request.headers['X-Line-Signature']
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    user_id = event.source.user_id
    msg = event.message.text
    if user_id not in user_state:
        user_state[user_id] = {'step': -2, 'data': []}

        introduction = ("您好，我是健康智能管家。\n"
                 "也是糖尿病預測專家，\n"
                 "您可以叫我阿瑄=U=\n"
                 "請問是否進行糖尿病預測呢?")

        buttons_template = TemplateSendMessage(
            alt_text='是否進行糖尿病預測？',
            template=ButtonsTemplate(
                title='是否進行糖尿病預測？',
                text=introduction,
                actions=[
                    PostbackAction(label='是', data='action=yes'),
                    PostbackAction(label='否', data='action=no')
                ]
            )
        )

        line_bot_api.reply_message(event.reply_token, buttons_template)
        return

    step = user_state[user_id]['step']

    if step < 4:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請選擇按鈕選項。"))
        return

    current_question_index = step - 4
    try:
        if current_question_index >= 0:  # 0 1 2 3 => button部分
            float_msg = float(msg)
            if float_msg < 0: #負數偵錯
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text="請輸入大於等於零的有效數字")
                )
                return
            user_state[user_id]['data'].append(float_msg)
            step += 1
            user_state[user_id]['step'] = step

            if step < 8:
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text=questions[current_question_index + 1])
                )
            else:
                process_final_input(event.reply_token, user_id)
        else: # 4 5 6 7 => text部分
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text="請選擇按鈕選項。")
            )
    except ValueError: #輸入非數字字元的偵錯
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="請輸入有效數字")
        )

#最後輸出
def process_final_input(reply_token, user_id):
    user_data = user_state[user_id]['data']
    user_input = pd.DataFrame([user_data], columns=[
        'gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
        'bmi', 'HbA1c_level', 'blood_glucose_level'
    ])


    prediction = model.predict(user_input)
    result = "没有糖尿病" if prediction[0] == 0 else "有糖尿病"

    prediction = model.predict_proba(user_input)[0]

    line_bot_api.reply_message(reply_token, [
        TextSendMessage(text=f"{result}"),
        TextSendMessage(text=f"糖尿病機率:{prediction[1]*100:.2f}%"),
        TextSendMessage(text="謝謝光臨!! 有需要都可以在叫我喔")
    ])
    del user_state[user_id]

@handler.add(PostbackEvent)
def handle_postback(event):
    user_id = event.source.user_id
    data = event.postback.data

    if user_id not in user_state:
        user_state[user_id] = {'step': -2, 'data': []}

    if data == 'action=yes':
        user_state[user_id] = {'step': 0, 'data': []}
        send_button_template(event.reply_token, 0)
    elif data == 'action=no':
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="謝謝光臨!! 有需要都可以在叫我喔")
        )
        del user_state[user_id]
    else:
        step = user_state[user_id]['step']
        user_state[user_id]['data'].append(int(data))
        step += 1
        user_state[user_id]['step'] = step
        if step < 4:
            send_button_template(event.reply_token, step)
        elif step == 4:
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text=questions[0]))
        else:
            # Handle the next text question
            current_question_index = step - 4
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text=questions[current_question_index]))

def send_button_template(reply_token, step):
    title, choices = button_questions[step]
    actions = [PostbackAction(label=label, data=data) for label, data in choices]
    template_message = TemplateSendMessage(
        alt_text=f'{title}',
        template=ButtonsTemplate(
            title=title, text=f'請選擇您的{title}', actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, template_message)
    step

if __name__ == "__main__":
    app.run(port=port)

 * ngrok tunnel "https://f03d-211-20-52-39.ngrok-free.app" -> "http://127.0.0.1:5000" 


C:\Users\lenovo\AppData\Local\Temp\ipykernel_6920\4282925020.py:21: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)  # token 確認
C:\Users\lenovo\AppData\Local\Temp\ipykernel_6920\4282925020.py:22: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(secret)      # secret 確認


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jul/2024 17:49:58] "POST / HTTP/1.1" 200 -
C:\Users\lenovo\AppData\Local\Temp\ipykernel_6920\4282925020.py:94: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, buttons_template)
127.0.0.1 - - [02/Jul/2024 17:50:13] "POST / HTTP/1.1" 200 -
C:\Users\lenovo\AppData\Local\Temp\ipykernel_6920\4282925020.py:196: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_mes